# Post-Silicon Semiconductor Data Analysis: CPU vs GPU Performance Comparison

This notebook demonstrates the performance difference between CPU and GPU computing for analyzing large-scale post-silicon semiconductor test data. We'll perform computationally intensive operations including data preprocessing, statistical analysis, and machine learning model training.

## Overview

Post-silicon validation generates massive amounts of test data from semiconductor chips. Analyzing this data requires significant computational resources. This notebook compares:

- **CPU Processing**: Traditional computing on your laptop
- **GPU Processing**: Accelerated computing using Google Colab's GPU

We'll measure execution time for identical operations and calculate the speedup achieved with GPU acceleration.

## 1. Environment Setup and Hardware Detection

First, we'll detect the available hardware (CPU or GPU) and install necessary packages. The notebook automatically adapts to the available hardware.

In [ ]:
import sys
import time
import platform
import warnings
warnings.filterwarnings('ignore')

print("=" * 70)
print("SYSTEM INFORMATION")
print("=" * 70)
print(f"Python Version: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"Processor: {platform.processor()}")
print("=" * 70)

In [ ]:
# Install required packages
!pip install -q numpy pandas scikit-learn matplotlib seaborn cupy-cuda12x 2>/dev/null || pip install -q numpy pandas scikit-learn matplotlib seaborn

In [ ]:
# Import libraries and detect GPU
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Try to import CuPy for GPU acceleration
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("✓ GPU DETECTED - CuPy successfully imported")
    print(f"✓ GPU Device: {cp.cuda.Device(0).compute_capability}")
    print(f"✓ GPU Memory: {cp.cuda.Device(0).mem_info[1] / 1e9:.2f} GB")
except ImportError:
    GPU_AVAILABLE = False
    print("✗ GPU NOT AVAILABLE - Using CPU only")
    print("  To enable GPU: Runtime -> Change runtime type -> GPU")

print("\n" + "=" * 70)
print(f"COMPUTE MODE: {'GPU ACCELERATED' if GPU_AVAILABLE else 'CPU ONLY'}")
print("=" * 70)

## 2. Load Semiconductor Test Data

We'll load the synthetic post-silicon semiconductor test data. This dataset contains 50,000 chips with 100 test parameters each, representing voltage, current, frequency, temperature, power, timing, leakage, and noise measurements.

In [ ]:
# Load data
print("Loading semiconductor test data...")
df = pd.read_csv('data/semiconductor_test_data.csv')

print(f"\nDataset Shape: {df.shape}")
print(f"Total Chips: {len(df):,}")
print(f"Total Features: {df.shape[1] - 4}")
print(f"\nPass/Fail Distribution:")
print(df['pass_fail'].value_counts())
print(f"\nMemory Usage: {df.memory_usage(deep=True).sum() / 1e6:.2f} MB")

# Display first few rows
df.head()

In [ ]:
# Visualize pass/fail distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart
colors = ['#90EE90', '#FFB6C6']  # Light green for PASS, light red for FAIL
df['pass_fail'].value_counts().plot(kind='pie', ax=axes[0], autopct='%1.1f%%', 
                                     colors=colors, startangle=90)
axes[0].set_ylabel('')
axes[0].set_title('Chip Test Results Distribution', fontsize=14, fontweight='bold')

# Bar chart
df['pass_fail'].value_counts().plot(kind='bar', ax=axes[1], color=colors)
axes[1].set_xlabel('Test Result', fontsize=12)
axes[1].set_ylabel('Count', fontsize=12)
axes[1].set_title('Chip Test Results Count', fontsize=14, fontweight='bold')
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

## 3. Data Preprocessing

Before analysis, we need to prepare the data by extracting numerical features and creating labels. We'll separate the feature columns from metadata columns.

In [ ]:
# Prepare data for analysis
# Extract numerical features (exclude chip_id, lot_id, wafer_id, pass_fail)
feature_cols = [col for col in df.columns if col not in ['chip_id', 'lot_id', 'wafer_id', 'pass_fail']]
X = df[feature_cols].values
y = (df['pass_fail'] == 'PASS').astype(int).values

print(f"Feature Matrix Shape: {X.shape}")
print(f"Label Vector Shape: {y.shape}")
print(f"\nFeature Matrix Size: {X.nbytes / 1e6:.2f} MB")

## 4. Performance Test 1: Matrix Operations

We'll perform large-scale matrix operations including correlation matrix computation and covariance calculations. These operations are fundamental in semiconductor data analysis for identifying parameter relationships.

### CPU Version: Matrix Operations

In [ ]:
print("=" * 70)
print("TEST 1: MATRIX OPERATIONS (CPU)")
print("=" * 70)

# Use NumPy for CPU computation
X_cpu = X.copy()

start_time = time.time()

# Compute correlation matrix
corr_matrix_cpu = np.corrcoef(X_cpu.T)

# Compute covariance matrix
cov_matrix_cpu = np.cov(X_cpu.T)

# Matrix multiplication
result_cpu = np.dot(X_cpu.T, X_cpu)

cpu_time_matrix = time.time() - start_time

print(f"\n✓ Correlation Matrix Shape: {corr_matrix_cpu.shape}")
print(f"✓ Covariance Matrix Shape: {cov_matrix_cpu.shape}")
print(f"✓ Matrix Multiplication Result Shape: {result_cpu.shape}")
print(f"\n⏱ CPU Execution Time: {cpu_time_matrix:.2f} seconds")
print("=" * 70)

### GPU Version: Matrix Operations

Now we'll perform the same operations using GPU acceleration with CuPy. If GPU is not available, this section will be skipped.

In [ ]:
if GPU_AVAILABLE:
    print("=" * 70)
    print("TEST 1: MATRIX OPERATIONS (GPU)")
    print("=" * 70)
    
    # Transfer data to GPU
    X_gpu = cp.array(X)
    
    start_time = time.time()
    
    # Compute correlation matrix on GPU
    corr_matrix_gpu = cp.corrcoef(X_gpu.T)
    
    # Compute covariance matrix on GPU
    cov_matrix_gpu = cp.cov(X_gpu.T)
    
    # Matrix multiplication on GPU
    result_gpu = cp.dot(X_gpu.T, X_gpu)
    
    # Synchronize GPU
    cp.cuda.Stream.null.synchronize()
    
    gpu_time_matrix = time.time() - start_time
    
    print(f"\n✓ Correlation Matrix Shape: {corr_matrix_gpu.shape}")
    print(f"✓ Covariance Matrix Shape: {cov_matrix_gpu.shape}")
    print(f"✓ Matrix Multiplication Result Shape: {result_gpu.shape}")
    print(f"\n⏱ GPU Execution Time: {gpu_time_matrix:.2f} seconds")
    print(f"🚀 Speedup: {cpu_time_matrix / gpu_time_matrix:.2f}x faster")
    print("=" * 70)
else:
    print("\n⚠ GPU not available. Skipping GPU matrix operations.")
    print("  Enable GPU in Colab: Runtime -> Change runtime type -> GPU")
    gpu_time_matrix = None

## 5. Performance Test 2: Statistical Analysis

We'll perform comprehensive statistical analysis including mean, standard deviation, percentiles, and other statistical measures across all test parameters. This is crucial for understanding the distribution of semiconductor test results.

### CPU Version: Statistical Analysis

In [ ]:
print("=" * 70)
print("TEST 2: STATISTICAL ANALYSIS (CPU)")
print("=" * 70)

start_time = time.time()

# Compute various statistics
mean_cpu = np.mean(X_cpu, axis=0)
std_cpu = np.std(X_cpu, axis=0)
median_cpu = np.median(X_cpu, axis=0)
percentile_25_cpu = np.percentile(X_cpu, 25, axis=0)
percentile_75_cpu = np.percentile(X_cpu, 75, axis=0)
min_cpu = np.min(X_cpu, axis=0)
max_cpu = np.max(X_cpu, axis=0)

cpu_time_stats = time.time() - start_time

print(f"\n✓ Computed statistics for {len(feature_cols)} features")
print(f"✓ Mean values shape: {mean_cpu.shape}")
print(f"✓ Std deviation shape: {std_cpu.shape}")
print(f"\n⏱ CPU Execution Time: {cpu_time_stats:.2f} seconds")
print("=" * 70)

### GPU Version: Statistical Analysis

In [ ]:
if GPU_AVAILABLE:
    print("=" * 70)
    print("TEST 2: STATISTICAL ANALYSIS (GPU)")
    print("=" * 70)
    
    start_time = time.time()
    
    # Compute various statistics on GPU
    mean_gpu = cp.mean(X_gpu, axis=0)
    std_gpu = cp.std(X_gpu, axis=0)
    median_gpu = cp.median(X_gpu, axis=0)
    percentile_25_gpu = cp.percentile(X_gpu, 25, axis=0)
    percentile_75_gpu = cp.percentile(X_gpu, 75, axis=0)
    min_gpu = cp.min(X_gpu, axis=0)
    max_gpu = cp.max(X_gpu, axis=0)
    
    # Synchronize GPU
    cp.cuda.Stream.null.synchronize()
    
    gpu_time_stats = time.time() - start_time
    
    print(f"\n✓ Computed statistics for {len(feature_cols)} features")
    print(f"✓ Mean values shape: {mean_gpu.shape}")
    print(f"✓ Std deviation shape: {std_gpu.shape}")
    print(f"\n⏱ GPU Execution Time: {gpu_time_stats:.2f} seconds")
    print(f"🚀 Speedup: {cpu_time_stats / gpu_time_stats:.2f}x faster")
    print("=" * 70)
else:
    print("\n⚠ GPU not available. Skipping GPU statistical analysis.")
    gpu_time_stats = None

## 6. Performance Test 3: Principal Component Analysis (PCA)

PCA is used in semiconductor analysis to reduce the dimensionality of test data while preserving the most important variance. This helps identify the key parameters that distinguish passing from failing chips.

### CPU Version: PCA

In [ ]:
print("=" * 70)
print("TEST 3: PRINCIPAL COMPONENT ANALYSIS (CPU)")
print("=" * 70)

start_time = time.time()

# Standardize features
scaler_cpu = StandardScaler()
X_scaled_cpu = scaler_cpu.fit_transform(X_cpu)

# Apply PCA
pca_cpu = PCA(n_components=20)
X_pca_cpu = pca_cpu.fit_transform(X_scaled_cpu)

cpu_time_pca = time.time() - start_time

print(f"\n✓ Original dimensions: {X_cpu.shape}")
print(f"✓ Reduced dimensions: {X_pca_cpu.shape}")
print(f"✓ Explained variance ratio: {pca_cpu.explained_variance_ratio_.sum():.4f}")
print(f"\n⏱ CPU Execution Time: {cpu_time_pca:.2f} seconds")
print("=" * 70)

### GPU Version: PCA

For GPU-accelerated PCA, we'll use CuPy for matrix operations. Note that scikit-learn's PCA doesn't directly support GPU, but the underlying matrix operations can be accelerated.

In [ ]:
if GPU_AVAILABLE:
    print("=" * 70)
    print("TEST 3: PRINCIPAL COMPONENT ANALYSIS (GPU)")
    print("=" * 70)
    
    start_time = time.time()
    
    # Standardize features on GPU
    mean_gpu_pca = cp.mean(X_gpu, axis=0)
    std_gpu_pca = cp.std(X_gpu, axis=0)
    X_scaled_gpu = (X_gpu - mean_gpu_pca) / std_gpu_pca
    
    # Compute covariance matrix on GPU
    cov_matrix_pca = cp.cov(X_scaled_gpu.T)
    
    # Compute eigenvalues and eigenvectors on GPU
    eigenvalues, eigenvectors = cp.linalg.eigh(cov_matrix_pca)
    
    # Sort by eigenvalues (descending)
    idx = cp.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    # Select top 20 components
    components_gpu = eigenvectors[:, :20]
    X_pca_gpu = cp.dot(X_scaled_gpu, components_gpu)
    
    # Synchronize GPU
    cp.cuda.Stream.null.synchronize()
    
    gpu_time_pca = time.time() - start_time
    
    explained_variance = eigenvalues[:20] / cp.sum(eigenvalues)
    
    print(f"\n✓ Original dimensions: {X_gpu.shape}")
    print(f"✓ Reduced dimensions: {X_pca_gpu.shape}")
    print(f"✓ Explained variance ratio: {cp.sum(explained_variance).get():.4f}")
    print(f"\n⏱ GPU Execution Time: {gpu_time_pca:.2f} seconds")
    print(f"🚀 Speedup: {cpu_time_pca / gpu_time_pca:.2f}x faster")
    print("=" * 70)
else:
    print("\n⚠ GPU not available. Skipping GPU PCA.")
    gpu_time_pca = None

## 7. Performance Test 4: Machine Learning Model Training

We'll train a Random Forest classifier to predict chip pass/fail status based on test parameters. This represents a real-world semiconductor analysis task where we need to identify patterns that distinguish good chips from defective ones.

In [ ]:
print("=" * 70)
print("TEST 4: MACHINE LEARNING MODEL TRAINING")
print("=" * 70)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_cpu, y, test_size=0.2, random_state=42)

print(f"\nTraining set size: {X_train.shape[0]:,}")
print(f"Test set size: {X_test.shape[0]:,}")

# Train Random Forest (CPU-based)
start_time = time.time()

rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

cpu_time_ml = time.time() - start_time

# Evaluate model
y_pred = rf_model.predict(X_test)
accuracy = (y_pred == y_test).mean()

print(f"\n✓ Model trained successfully")
print(f"✓ Model accuracy: {accuracy:.4f}")
print(f"\n⏱ Training Time: {cpu_time_ml:.2f} seconds")
print("=" * 70)

# Note: Random Forest in scikit-learn doesn't have direct GPU support
# For GPU-accelerated ML, libraries like XGBoost or cuML would be used
# This test demonstrates CPU-only ML workload
gpu_time_ml = None

## 8. Performance Summary and Comparison

Let's summarize all the performance tests and visualize the speedup achieved with GPU acceleration.

In [ ]:
# Create performance summary
results = {
    'Test': ['Matrix Operations', 'Statistical Analysis', 'PCA', 'ML Training'],
    'CPU Time (s)': [cpu_time_matrix, cpu_time_stats, cpu_time_pca, cpu_time_ml],
    'GPU Time (s)': [gpu_time_matrix, gpu_time_stats, gpu_time_pca, gpu_time_ml]
}

results_df = pd.DataFrame(results)

# Calculate speedup where GPU is available
results_df['Speedup'] = results_df.apply(
    lambda row: row['CPU Time (s)'] / row['GPU Time (s)'] if row['GPU Time (s)'] is not None else None,
    axis=1
)

print("=" * 70)
print("PERFORMANCE SUMMARY")
print("=" * 70)
print(results_df.to_string(index=False))
print("=" * 70)

if GPU_AVAILABLE:
    total_cpu_time = results_df['CPU Time (s)'].sum()
    total_gpu_time = results_df['GPU Time (s)'].dropna().sum()
    overall_speedup = total_cpu_time / total_gpu_time
    time_saved = total_cpu_time - total_gpu_time
    
    print(f"\n📊 OVERALL PERFORMANCE:")
    print(f"   Total CPU Time: {total_cpu_time:.2f} seconds")
    print(f"   Total GPU Time: {total_gpu_time:.2f} seconds")
    print(f"   Overall Speedup: {overall_speedup:.2f}x faster")
    print(f"   Time Saved: {time_saved:.2f} seconds ({time_saved/60:.2f} minutes)")
    print(f"   Efficiency Gain: {(1 - total_gpu_time/total_cpu_time)*100:.1f}%")
    print("=" * 70)
else:
    print("\n⚠ Run this notebook on Google Colab with GPU enabled to see speedup!")
    print("  Instructions: Runtime -> Change runtime type -> GPU")

In [ ]:
# Visualize performance comparison
if GPU_AVAILABLE:
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Bar chart comparing CPU vs GPU times
    tests = results_df['Test']
    x = np.arange(len(tests))
    width = 0.35
    
    bars1 = axes[0].bar(x - width/2, results_df['CPU Time (s)'], width, 
                        label='CPU', color='#FFB6C6', alpha=0.8)
    bars2 = axes[0].bar(x + width/2, results_df['GPU Time (s)'].fillna(0), width, 
                        label='GPU', color='#ADD8E6', alpha=0.8)
    
    axes[0].set_xlabel('Test Type', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Execution Time (seconds)', fontsize=12, fontweight='bold')
    axes[0].set_title('CPU vs GPU Execution Time Comparison', fontsize=14, fontweight='bold')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(tests, rotation=45, ha='right')
    axes[0].legend()
    axes[0].grid(axis='y', alpha=0.3)
    
    # Speedup chart
    speedup_data = results_df[results_df['Speedup'].notna()]
    bars = axes[1].bar(speedup_data['Test'], speedup_data['Speedup'], 
                       color='#90EE90', alpha=0.8)
    axes[1].axhline(y=1, color='red', linestyle='--', linewidth=2, label='No Speedup')
    axes[1].set_xlabel('Test Type', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Speedup Factor (x)', fontsize=12, fontweight='bold')
    axes[1].set_title('GPU Speedup Over CPU', fontsize=14, fontweight='bold')
    axes[1].set_xticklabels(speedup_data['Test'], rotation=45, ha='right')
    axes[1].legend()
    axes[1].grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        axes[1].text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.2f}x',
                    ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    # Show only CPU times when GPU is not available
    fig, ax = plt.subplots(figsize=(10, 6))
    
    bars = ax.bar(results_df['Test'], results_df['CPU Time (s)'], 
                  color='#FFB6C6', alpha=0.8)
    ax.set_xlabel('Test Type', fontsize=12, fontweight='bold')
    ax.set_ylabel('Execution Time (seconds)', fontsize=12, fontweight='bold')
    ax.set_title('CPU Execution Time (GPU Not Available)', fontsize=14, fontweight='bold')
    ax.set_xticklabels(results_df['Test'], rotation=45, ha='right')
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 TIP: Enable GPU in Google Colab to see dramatic speedup!")
    print("   Runtime -> Change runtime type -> Hardware accelerator -> GPU")

## 9. Key Findings and Insights

### Performance Analysis

The performance comparison demonstrates the significant computational advantages of GPU acceleration for large-scale semiconductor data analysis:

1. **Matrix Operations**: GPU excels at parallel matrix computations, showing substantial speedup for correlation and covariance calculations.

2. **Statistical Analysis**: Batch statistical operations benefit greatly from GPU's parallel processing capabilities.

3. **PCA**: Dimensionality reduction through PCA involves heavy matrix operations, making it ideal for GPU acceleration.

4. **Machine Learning**: While Random Forest in scikit-learn is CPU-optimized, GPU-based ML libraries (XGBoost, cuML) can provide additional speedup.

### When to Use GPU

GPU acceleration is most beneficial when:
- Working with large datasets (>10,000 samples)
- Performing matrix-heavy operations
- Running iterative algorithms
- Processing multiple batches in parallel

### Practical Implications for Semiconductor Analysis

For post-silicon validation engineers:
- **Faster iteration**: Quickly test multiple analysis approaches
- **Real-time analysis**: Process test data as it's generated
- **Scalability**: Handle larger datasets without proportional time increase
- **Cost efficiency**: Reduce compute time and associated costs

## 10. Conclusion

This notebook demonstrated the performance difference between CPU and GPU computing for semiconductor data analysis. Key takeaways:

- ✅ GPU acceleration provides significant speedup for matrix operations and statistical analysis
- ✅ Google Colab's free GPU makes advanced computing accessible
- ✅ The same code can run on both CPU and GPU with minimal modifications
- ✅ For production semiconductor analysis, GPU acceleration can save hours of computation time

### Next Steps

To further optimize your semiconductor data analysis:
1. Explore GPU-accelerated ML libraries (XGBoost, cuML)
2. Implement batch processing for even larger datasets
3. Use mixed precision computing for additional speedup
4. Consider distributed computing for multi-GPU setups